# Code:Nation - Desafio: Descubra as melhores notas de matemática do ENEM 2016

O contexto do desafio gira em torno dos resultados do ENEM 2016 (disponíveis no arquivo train.csv). Este arquivo, e apenas ele, deve ser utilizado para todos os desafios. Qualquer dúvida a respeito das colunas, consulte o [Dicionário dos Microdados do Enem 2016](https://s3-us-west-1.amazonaws.com/acceleration-assets-highway/data-science/dicionario-de-dados.zip).

No arquivo test.csv crie um modelo para prever nota da prova de matemática (coluna `NU_NOTA_MT`) de quem participou do ENEM 2016. 

Salve sua resposta em um arquivo chamado answer.csv com duas colunas: `NU_INSCRICAO` e `NU_NOTA_MT`.

In [1]:
import sklearn
print("A versão do scikit-learn é ", sklearn.__version__)
if sklearn.__version__ >= '0.18':
    print("Tudo certo!")
else:
    print("Você precisa fazer upgrade do scikit-learn ou ficar atento com as diferenças das versões")
    print("Pode ser feito executando:\n")
    print("pip install scikit-learn==0.18.1")

A versão do scikit-learn é  0.19.2
Tudo certo!


In [5]:
# Importar as bibliotecas necessárias para este projeto
import numpy as np
import pandas as pd
#import visuals as vs # Supplementary code
from sklearn.model_selection import ShuffleSplit

# Formatação mais bonita para os notebooks
%matplotlib inline

# Executar o conjunto de dados de imóveis de Boston
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')
target = df_train['NU_NOTA_MT']
feature = pd.DataFrame(df_train['TX_GABARITO_MT'])
    
# Êxito
print("O conjunto de microdados do ENEM 2016 tem {} pontos com {} variáveis em cada.".format(*df_train.shape))

O conjunto de microdados do ENEM 2016 tem 13730 pontos com 167 variáveis em cada.


In [10]:
df_train.head()

,Unnamed: 0,NU_INSCRICAO,NU_ANO,CO_MUNICIPIO_RESIDENCIA,NO_MUNICIPIO_RESIDENCIA,CO_UF_RESIDENCIA,SG_UF_RESIDENCIA,NU_IDADE,TP_SEXO,TP_ESTADO_CIVIL,...,Q041,Q042,Q043,Q044,Q045,Q046,Q047,Q048,Q049,Q050
0,1,ed50e8aaa58e7a806c337585efee9ca41f1eb1ad,2016,4314902,Porto Alegre,43,RS,24,M,0.0,...,5.0,A,A,A,A,A,A,A,B,D
1,2,2c3acac4b33ec2b195d77e7c04a2d75727fad723,2016,2304707,Granja,23,CE,17,F,0.0,...,NaN,A,A,C,A,B,A,A,C,A
2,3,f4545f8ccb9ff5c8aad7d32951b3f251a26e6568,2016,2304400,Fortaleza,23,CE,21,F,0.0,...,NaN,A,A,A,A,C,A,A,B,A
3,4,3d6ec248fef899c414e77f82d5c6d2bffbeaf7fe,2016,3304557,Rio de Janeiro,33,RJ,25,F,0.0,...,5.0,C,A,A,A,A,D,A,A,A
4,5,bf896ac8d3ecadd6dba1dfbf50110afcbf5d3268,2016,1302603,Manaus,13,AM,28,M,0.0,...,NaN,A,A,A,A,A,A,A,A,A


In [12]:
df_test.columns

Index(['NU_INSCRICAO', 'CO_UF_RESIDENCIA', 'SG_UF_RESIDENCIA', 'NU_IDADE',
       'TP_SEXO', 'TP_COR_RACA', 'TP_NACIONALIDADE', 'TP_ST_CONCLUSAO',
       'TP_ANO_CONCLUIU', 'TP_ESCOLA', 'TP_ENSINO', 'IN_TREINEIRO',
       'TP_DEPENDENCIA_ADM_ESC', 'IN_BAIXA_VISAO', 'IN_CEGUEIRA', 'IN_SURDEZ',
       'IN_DISLEXIA', 'IN_DISCALCULIA', 'IN_SABATISTA', 'IN_GESTANTE',
       'IN_IDOSO', 'TP_PRESENCA_CN', 'TP_PRESENCA_CH', 'TP_PRESENCA_LC',
       'CO_PROVA_CN', 'CO_PROVA_CH', 'CO_PROVA_LC', 'CO_PROVA_MT',
       'NU_NOTA_CN', 'NU_NOTA_CH', 'NU_NOTA_LC', 'TP_LINGUA',
       'TP_STATUS_REDACAO', 'NU_NOTA_COMP1', 'NU_NOTA_COMP2', 'NU_NOTA_COMP3',
       'NU_NOTA_COMP4', 'NU_NOTA_COMP5', 'NU_NOTA_REDACAO', 'Q001', 'Q002',
       'Q006', 'Q024', 'Q025', 'Q026', 'Q027', 'Q047'],
      dtype='object')

0    DADCCECBDECADCBACDBCEEABDDAABDBCEADDBECEBAEBC
1    DADCCECBDECADCBACDBCEEABDDAABDBCEADDBECEBAEBC
5    CEECDBDADEBCCEBABECEACBDABDBDDABDDECADCCEACBA
6    CCEEBCCEBAABDDECADCEECDBCBDBECBADDACEAABDBDAD
7    CBAACCEABDDCDBDDCEAECADBECEBACBDABDBEBCDADCEE
Name: TX_GABARITO_MT, dtype: object

In [7]:
list(feature[0])

['D',
 'A',
 'D',
 'C',
 'C',
 'E',
 'C',
 'B',
 'D',
 'E',
 'C',
 'A',
 'D',
 'C',
 'B',
 'A',
 'C',
 'D',
 'B',
 'C',
 'E',
 'E',
 'A',
 'B',
 'D',
 'D',
 'A',
 'A',
 'B',
 'D',
 'B',
 'C',
 'E',
 'A',
 'D',
 'D',
 'B',
 'E',
 'C',
 'E',
 'B',
 'A',
 'E',
 'B',
 'C']

In [8]:
dummies = pd.get_dummies(df_train['TX_GABARITO_MT'])
dummies

,BCABBDDBDCDCCCACEDCBADEEEECAACEACEBCBDEBBBDDD,BDDDBDEBBCEBCEACAACEEEEACEDCBADCCDCDCDBABBDBC,BDEBBCAACCBADCCACEDCEBCBDDDEEEEBCEADCDCDBABBD,CAACDBCEBCBDDDEEEEBCABBDBDEBBEADCDCCCACEDCBAD,CBAACCEABDDCDBDDCEAECADBECEBACBDABDBEBCDADCEE,CCEEBCCEBAABDDECADCEECDBCBDBECBADDACEAABDBDAD,CEECDBDADEBCCEBABECEACBDABDBDDABDDECADCCEACBA,DADCCECBDECADCBACDBCEEABDDAABDBCEADDBECEBAEBC
0,0,0,0,0,0,0,0,1
1,0,0,0,0,0,0,0,1
2,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,1,0
6,0,0,0,0,0,1,0,0
7,0,0,0,0,1,0,0,0
8,0,0,0,0,0,1,0,0
9,0,0,0,0,0,0,0,0
